In [1]:
pip uninstall tensorflow

Note: you may need to restart the kernel to use updated packages.


'D:\Users\divyakumar.prajapati\Anaconda' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
pip install tensorflow==2.1.0

Note: you may need to restart the kernel to use updated packages.


'D:\Users\divyakumar.prajapati\Anaconda' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import tensorflow as tf

from collections import defaultdict
import pandas as pd
import numpy as np
import os
import json
import pickle
import sys
import random
import glob
from sklearn import metrics,model_selection
import csv
from collections import Counter
import warnings
import string
import re
import nltk
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


In [2]:
MAX_DOCUMENT_LENGTH=100#128#100
maxlen_word=20#5#20
num_classes=4
embedding_dims=100
filters=250
kernel_size = 3
hidden_dims = 100
batch_size=128#5

In [3]:
ann_dir= r"D:\Users\divyakumar.prajapati\Downloads\train_sample.csv" #set_combined_final

duplicate_samples=1
duplicate_samples_pos=1
#n_files=60
n_epoch=30
f= open("report_sections_cnn_only_text_09_01_17_files_"+"_epoch_"+str(n_epoch)+".txt","w")
#f.write("Number of files: ")
#f.write(str(n_files))
f.write("\nNumber of epoch: ")
f.write(str(n_epoch))
f.write("\n")
load_from_vector=False
model_dir="cnn_only_text_section_header/"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [4]:
sh_dataset = defaultdict(lambda : None)
sh_dataset['target_names'] = ['Heading','Section Heading','Sub - Heading','Sub-Sub Heading']
sh_dataset['target'] =[]
sh_dataset['data'] =[]
sh_dataset['layout'] =[]

In [5]:
with open(ann_dir, 'rt',encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for i in range(duplicate_samples):
            if row['levels'] =="Heading":
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(0)
                    sh_dataset['data'].append(row['Text'])
            elif row['levels'] =="Section Heading":
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(1)  
                    sh_dataset['data'].append(row['Text'])
            elif row['levels'] =="Sub - Heading":
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(2)  
                    sh_dataset['data'].append(row['Text'])
            elif row['levels'] =="Sub-Sub Heading":
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(3)  
                    sh_dataset['data'].append(row['Text'])

In [6]:
# training data
training_dataset = defaultdict(lambda : None)
training_dataset['target_names'] =['Heading','Section Heading','Sub - Heading','Sub-Sub Heading']
training_dataset['target'] =[]
training_dataset['data'] =[]

In [7]:
len(sh_dataset['data'])

10057

In [8]:
pos_sample = min(sh_dataset['target'].count(0),sh_dataset['target'].count(1),sh_dataset['target'].count(2),sh_dataset['target'].count(3))

count_top_sample=0
count_sub_sample=0
count_subsub_sample=0

In [9]:
for i in range(len(sh_dataset['target'])):
    if sh_dataset['target'][i]==0:
        if count_top_sample<pos_sample:
            training_dataset['data'].append(sh_dataset['data'][i])
            training_dataset['target'].append(sh_dataset['target'][i])
            count_top_sample+=1

    elif sh_dataset['target'][i]==1:
        if count_sub_sample<pos_sample:
            training_dataset['data'].append(sh_dataset['data'][i])
            training_dataset['target'].append(sh_dataset['target'][i])
            count_sub_sample+=1
  
    elif sh_dataset['target'][i]==2:
        if count_sub_sample<pos_sample:
            training_dataset['data'].append(sh_dataset['data'][i])
            training_dataset['target'].append(sh_dataset['target'][i])
            count_sub_sample+=1
    else:
        if count_subsub_sample<pos_sample:
            training_dataset['data'].append(sh_dataset['data'][i])
            training_dataset['target'].append(sh_dataset['target'][i])
            count_subsub_sample+=1


In [10]:
training_dataset['target'].count(1)

635

In [11]:
training_dataset_ran = defaultdict(lambda : None)
training_dataset_ran['target_names'] =['Heading','Section Heading','Sub - Heading','Sub-Sub Heading']
training_dataset_ran['target'] =[]
training_dataset_ran['data'] =[]

In [12]:
import random
ran_index= random.sample(range(0, len(training_dataset['data'])), len(training_dataset['data']))
for i in ran_index:
    training_dataset_ran['data'].append(training_dataset['data'][i])
    training_dataset_ran['target'].append(training_dataset['target'][i])

In [13]:
del training_dataset   
del sh_dataset

In [14]:
print("Training")
f.write("Training\n")
print("Heading",training_dataset_ran["target"].count(0))
f.write("Top samples ")
f.write(str(training_dataset_ran["target"].count(1)))
f.write("\n")
print("Section Heading",training_dataset_ran["target"].count(1))
f.write("Top samples ")
f.write(str(training_dataset_ran["target"].count(1)))
f.write("\n")
print("Sub - Heading",training_dataset_ran["target"].count(2))
f.write("Sub samples ")
f.write(str(training_dataset_ran["target"].count(2)))
f.write("\n")
print("Sub-Sub Heading",training_dataset_ran["target"].count(3))
f.write("Subsub samples ")
f.write(str(training_dataset_ran["target"].count(3)))
f.write("\n")

Training
Heading 1435
Section Heading 635
Sub - Heading 800
Sub-Sub Heading 1435


1

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training_dataset_ran['data'],training_dataset_ran['target'], test_size=0.2)

In [16]:
df_train=pd.DataFrame({'line':X_train,'class':y_train})
df_train.to_csv(model_dir+"stratified_training_50", sep='\t',index=False, encoding='utf-8')

df_test=pd.DataFrame({'line':X_test,'class':y_test})
df_test.to_csv(model_dir+"stratified_test_10", sep='\t',index=False, encoding='utf-8')

In [17]:
# Process vocabulary character level
char_processor = learn.preprocessing.ByteProcessor(MAX_DOCUMENT_LENGTH)
X_train_char = np.array(list(char_processor.fit_transform(X_train)))
X_test_char = np.array(list(char_processor.transform(X_test)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [18]:
# Process vocabulary world level
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(maxlen_word,min_frequency=100)
X_train_word = np.array(list(vocab_processor.fit_transform(X_train)))
X_test_word = np.array(list(vocab_processor.transform(X_test)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [19]:
# #Class labels to categorical value
# y_train = keras.utils.to_categorical(y_train,num_classes=num_classes)
# y_test = keras.utils.to_categorical(y_test,num_classes=num_classes)

In [20]:
max_words = len(vocab_processor.vocabulary_)
max_char = 256

In [21]:
print(max_words)
print(max_char)

15
256


In [22]:
# char_CNN
input_char =  layers.Input(shape=(MAX_DOCUMENT_LENGTH,), name = 'input_char')

embedding_char= layers.Embedding(max_char,embedding_dims,input_length=MAX_DOCUMENT_LENGTH)(input_char)
dropout1_char = layers.Dropout(0.2)(embedding_char)

conv1D_char = layers.Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(dropout1_char)

globalMaxPooling1_char= layers.GlobalMaxPooling1D()(conv1D_char)

dense_hidden_char = layers.Dense(hidden_dims)(globalMaxPooling1_char)
dropout2_char = layers.Dropout(0.2)(dense_hidden_char)
activation_char= layers.Activation('relu')(dropout2_char)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [23]:
main_output = layers.Dense(num_classes, activation='softmax', name='main_output')(activation_char)

In [24]:
model = models.Model(inputs=[input_char], outputs=[main_output])

In [25]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_char (InputLayer)      (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 100)          25600     
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 250)           75250     
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               25100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
__________

In [26]:
filepath=model_dir+"weights-improvement-{epoch:02d}-{val_acc:.9f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = callbacks.History()

In [27]:
model.fit(X_train_char, y_train,
          batch_size=batch_size,shuffle=True,
          epochs=n_epoch,verbose=1,
          validation_data=(X_test_char, y_test),callbacks=[checkpoint,history])

Train on 3444 samples, validate on 861 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/30
3328/3444 [===========================>..] - ETA: 0s - loss: 1.1395 - acc: 0.5532
Epoch 00001: val_acc improved from -inf to 0.61556, saving model to cnn_only_text_section_header/weights-improvement-01-0.615563273.hdf5
3444/3444 [==============================] - 7s 2ms/sample - loss: 1.1350 - acc: 0.5528 - val_loss: 0.8815 - val_acc: 0.6156
Epoch 2/30
3328/3444 [===========================>..] - ETA: 0s - loss: 0.8336 - acc: 0.6475
Epoch 00002: val_acc improved from 0.61556 to 0.69686, saving model to cnn_only_text_section_header/weights-improvement-02-0.696864128.hdf5
3444/3444 [==============================] - 7s 2ms/sample - loss: 0.8332 - acc: 0.6478 - val_loss: 0.7448 - val_acc: 0.6969
Epoch 3/30
3328/3444 [===========================>..] - ETA: 0s - loss: 0.7274 - acc: 0.6941
Epoch 00003: val_acc 

In [28]:
#confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
yhat_probs = model.predict(X_test_char, verbose=0)

In [29]:
y_pred = []
for i in np.arange(len(yhat_probs)):
    y_pred.append(np.argmax(yhat_probs[i]))

In [30]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[281,   6,   2,   4],
       [  2,  93,  16,  20],
       [  2,  26,  57,  70],
       [  0,  18,  22, 242]])

In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       293
           1       0.65      0.71      0.68       131
           2       0.59      0.37      0.45       155
           3       0.72      0.86      0.78       282

    accuracy                           0.78       861
   macro avg       0.74      0.72      0.72       861
weighted avg       0.78      0.78      0.77       861



In [33]:
model.load_weights('/content/cnn_only_text_section_header/weights-improvement-28-0.787456453.hdf5')

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       293
           1       0.65      0.71      0.68       131
           2       0.59      0.37      0.45       155
           3       0.72      0.86      0.78       282

    accuracy                           0.78       861
   macro avg       0.74      0.72      0.72       861
weighted avg       0.78      0.78      0.77       861



In [ ]:
X_test

In [ ]:
testt = pd.read_csv('data/validation_file.csv')

In [ ]:
X_test_charr = np.array(list(char_processor.transform(testt.Text.tolist())))

In [ ]:
yhat_probss = model.predict(X_test_charr, verbose=0)

In [ ]:
y_predd = []
for i in np.arange(len(yhat_probss)):
    y_predd.append(np.argmax(yhat_probss[i]))

In [ ]:
tst = testt.Heading_level.tolist()
for n, i in enumerate(tst):
    if i == 1:
        tst[n] = 0
    elif i == 2:
        tst[n] = 1
    elif i == 3:
        tst[n] = 2
    else:
        tst[n] = 3

In [ ]:
cm = confusion_matrix(tst, y_predd)
cm

In [ ]:
print(classification_report(tst, y_predd))

Bert model

In [19]:
pip install ktrain

Note: you may need to restart the kernel to use updated packages.


'D:\Users\divyakumar.prajapati\Anaconda' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
import ktrain
from ktrain import text

In [18]:
train = pd.DataFrame()
train['text'] = training_dataset_ran['data']
train['target'] = training_dataset_ran['target']

In [19]:
import numpy as np

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X = np.array(train['target'])
c = enc.fit_transform([X])

In [20]:
dummies1 = pd.get_dummies(train.iloc[:, 1])

In [21]:
train_s = pd.DataFrame()
train_s['text'] = train['text']
train_s['0'] = dummies1.iloc[:, 0]
train_s['1'] = dummies1.iloc[:, 1]
train_s['2'] = dummies1.iloc[:, 2]
train_s['3'] = dummies1.iloc[:, 3]

In [24]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train_s,text_column='text', preprocess_mode='bert', maxlen=128, label_columns=['0','1','2','3'])

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [25]:
learner = ktrain.get_learner(text.text_classifier('bert', (x_train, y_train), preproc=preproc, multilabel=True), train_data=(x_train,y_train), val_data=(x_test, y_test), batch_size=32)

Is Multi-Label? True
maxlen is 128
done.


In [26]:
learner.fit(0.001,2)

Train on 3874 samples, validate on 431 samples
Epoch 1/2
  32/3874 [..............................] - ETA: 2:15:39

ResourceExhaustedError:  OOM when allocating tensor with shape[32,128,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node model_1/Encoder-6-MultiHeadSelfAttention-Norm/mul (defined at D:\Users\divyakumar.prajapati\Anaconda 3\lib\site-packages\keras_layer_normalization\layer_normalization.py:95) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_37131]

Errors may have originated from an input operation.
Input Source operations connected to node model_1/Encoder-6-MultiHeadSelfAttention-Norm/mul:
 model_1/Encoder-6-MultiHeadSelfAttention-Norm/truediv (defined at D:\Users\divyakumar.prajapati\Anaconda 3\lib\site-packages\keras_layer_normalization\layer_normalization.py:93)

Function call stack:
distributed_function


In [35]:
from sklearn.metrics import classification_report, confusion_matrix


In [59]:
x_test

[array([[  101,  2321,  1012, ...,     0,     0,     0],
        [  101,  5636,  4761, ...,     0,     0,     0],
        [  101,  1018,  1012, ...,     0,     0,     0],
        ...,
        [  101, 23614,  2224, ...,     0,     0,     0],
        [  101,  3424,  3597, ...,     0,     0,     0],
        [  101,  3447,  7047, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [72]:
learner.model.save_weights('weigths.h5')

In [24]:
learner.model.load_weights('weigths.h5')

In [26]:
c = learner.val_data

In [28]:
c[:-1]

([array([[ 101, 4004, 2679, ...,    0,    0,    0],
         [ 101, 2177, 1015, ...,    0,    0,    0],
         [ 101, 1022, 1012, ...,    0,    0,    0],
         ...,
         [ 101, 2143, 5435, ...,    0,    0,    0],
         [ 101, 2060, 9556, ...,    0,    0,    0],
         [ 101, 6643, 2098, ...,    0,    0,    0]]),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])],)

In [36]:
y_pred = learner.predict()

In [37]:
y_pred

array([[0.26339096, 0.17766337, 0.17027742, 0.36930683],
       [0.26339093, 0.17766337, 0.17027745, 0.36930674],
       [0.2633909 , 0.17766336, 0.1702774 , 0.36930683],
       ...,
       [0.26339096, 0.17766342, 0.17027733, 0.36930668],
       [0.26339093, 0.17766339, 0.17027737, 0.36930674],
       [0.263391  , 0.17766339, 0.17027736, 0.36930674]], dtype=float32)

In [42]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [101]:
def encode(c):
  for i in range(4):
    b = list(map(max, zip(*c)))
    if c[i][1] == b[1]:
      return c[i][0]
      break

In [102]:
Y_pred_h = []
for i in range(100):
  Y_pred_h.append(encode(predictor.predict(train['text'].loc[i])))

In [103]:
len(Y_pred_h)

100

In [104]:
for i in range(len(Y_pred_h)):
  Y_pred_h[i] = int(Y_pred_h[i])

In [105]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(Y_pred_h, train['target'].loc[:99])

In [106]:
cm

array([[ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [30, 21, 26, 23]])

In [108]:
print(classification_report(Y_pred_h, train['target'].loc[:99]))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       1.00      0.23      0.37       100

    accuracy                           0.23       100
   macro avg       0.25      0.06      0.09       100
weighted avg       1.00      0.23      0.37       100



In [90]:
c = [('0', 0.26339102), ('1', 0.47766337), ('2', 0.57027733), ('3', 0.26930683)]

In [96]:
b

['3', 0.57027733]

In [94]:
max(c)[1]

0.26930683